# Memory: A Deep Dive: Part II (Schema Based Autonomous Memory Generation)


## Overview
This code implements a workflow-based memory generation system that combines LangGraph and GPT-4 mini to process interactions into structured memories. The system takes predefined memory schemas (episodic, semantic, procedural, prospective) and generates contextual memories through a graph-based workflow, maintaining hierarchical organization (type → label → content → context).

## Motivation
Several key factors motivate this implementation:

1. **Knowledge Structure Automation**
   - Converting interactions into structured memories manually is time-consuming
   - Automated generation streamlines the memory creation process
   - LLMs can provide context-aware memory generation

2. **Flexible Control Flow**
   - Need for systems that manage memory generation workflow
   - Importance of state management during generation
   - Value of maintaining generation history

3. **Standardized Memory Organization**
   - Need for consistent memory structure
   - Importance of schema-based approaches
   - Benefits of hierarchical memory organization

4. **Interactive Development**
   - Recognition that memory generation needs controlled flow
   - Value of state tracking during generation
   - Importance of flexible execution in memory creation

## Key Components
1. **State Management System**: 
   - Maintains workflow state using Pydantic models
   - Tracks interaction, memory schema, labels, and memories
   - Ensures type safety and data validation

2. **LLM Integration**: 
   - Leverages OpenAI's GPT-4 mini model
   - Generates memories based on schema types:
     - Episodic: Experience-based memories
     - Semantic: Knowledge-based memories
     - Procedural: Process-based memories
     - Prospective: Future-oriented memories

3. **Graph-based Workflow**: 
   - Uses LangGraph's StateGraph for orchestration
   - Implements memory type determination node
   - Controls memory generation through edges

4. **Memory Structure**:
   - Follows hierarchical organization
   - Maintains context with each memory
   - Supports multiple memory types

## Method
The system follows this workflow:

1. **Initialization**:
   - Takes interaction and memory schema as input
   - Sets up state management system
   - Initializes LLM client

2. **Memory Type Determination**:
   - Analyzes interaction against schema
   - Selects relevant memory types
   - Prepares memory labels

3. **Memory Generation**:
   - Processes each memory label
   - Generates structured memories
   - Updates state with results

4. **Flow Control**:
   - Manages generation process
   - Controls workflow completion
   - Maintains generation state

## Visual Overview
A flowchart representing the design and flow of the workflow.

<div style="max-width:400px;">
    
![image.png](../images/memory_generation.png)
    
</div>

## Conclusion
This implementation demonstrates a practical approach to automated memory generation. The system combines graph-based workflow management with LLM capabilities, allowing for structured memory creation while maintaining clear process control.

Key advantages include:
- Structured memory generation workflow
- Clear state management
- Schema-based organization
- Modular node processing

Future improvements could focus on:
- Enhanced memory validation
- Error handling mechanisms
- State persistence strategies
- Workflow optimization techniques

This system provides a foundation for building more sophisticated memory generation systems, particularly in applications requiring structured knowledge creation and process control.

# Dependencies and Imports
Install dependencies and import libraries.

In [1]:
%%capture

!pip install langgraph
!pip install langgraph-sdk
!pip install langgraph-checkpoint-sqlite
!pip install langchain-community
!pip install langchain-core
!pip install langchain-openai

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, END
from langchain.schema import HumanMessage
from langchain_openai import ChatOpenAI

from pydantic import BaseModel
from typing import Optional

import os


## Clients
Import API keys and instantiate clients.

In [3]:
os.environ['OPENAI_API_KEY'] = 'YOUR-API-KEY'
llm = ChatOpenAI(model='gpt-4o-mini')

## Define Agent State
We'll define the state that our agent will maintain throughout its operation.


In [4]:
class State(BaseModel):
    interaction: str
    memory_schema: list[str] = []
    memory_labels: list[str] = []
    memories: list[str] = []

## Define Node Functions
Now we'll define the main node functions that our agent will use: generate_schema and update_instructions.


In [5]:
def memory_type_determination_node(state: State):
    ''' Determine Memory Types '''
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with selecting the types of memories to generate for the following interaction:'
        '{interaction}'
        'You must carefully choose N memory types from the following predetermined memory schema[# memory_type ## memory_label ### memory_description] :'
        '{memory_schema}'
        'The chosen memory types must be critical in the formation of valuable memories based on the interaction.'
        'The response must include only the memory_label and memory_description of the selected memory type. Do not include the meory_type.'
        'Response Format: ## memory_label ### memory_description|...|## memory_label ### memory_description'
    )
    message = HumanMessage(content=prompt.format(interaction=state.interaction, memory_schema=state.memory_schema))
    memory_labels = llm.invoke([message]).content.strip().split('|')

    state.memory_labels = memory_labels
    return state


def memory_generation_node(state: State):
    ''' Generate Memory '''
    memory_label = state.memory_labels[0]
    prompt = ChatPromptTemplate.from_template(
        'You are tasked with generating a memory for a given interaction and memory_label.'
        'Interaction:'
        '{interaction}'
        'Memory Type[## memory_label ### memory_description] :'
        '{memory_label}'
        'Response Format: # memory_label ## memory_content ### memory_context'
    )
    message = HumanMessage(content=prompt.format(interaction=state.interaction, memory_schema=state.memory_schema, memory_label=memory_label))
    memory = llm.invoke([message]).content.strip()
    
    print('Memory Generated:')
    print('-----------------')
    print(memory)
    
    state.memory_labels.pop(0)
    state.memories.append(memory)
    return state
    

## Define Edge Functions
Now we'll define the conditional edge function that our agent will use to control the workflow.

In [6]:
def generate_memory(state: State):
    if state.memory_labels:
        return 'memory_generation_node'
    else:
        return END
    

## Build Workflow
Now we'll create our workflow and compile it.


In [7]:
builder = StateGraph(State)

# Add nodes to the graph
builder.add_node('memory_type_determination_node', memory_type_determination_node)
builder.add_node('memory_generation_node', memory_generation_node)

# Add edges to the graph
builder.set_entry_point('memory_type_determination_node')
builder.add_conditional_edges('memory_type_determination_node', generate_memory)
builder.add_conditional_edges('memory_generation_node', generate_memory)

# Compile the graph
graph = builder.compile()

# Main Function
Define the function that runs the instanciates the workflow and its state.

In [8]:
def run_memory_generator(interaction: str, memory_schema: list):
    state = State(interaction=interaction, memory_schema=memory_schema)
    
    for output in graph.stream(state):
        pass

# Run Program
Instanciate the main function and observe outputs.

In [9]:
memory_schema = [
    '# episodic ## tutoring_sessions ### A record of individual tutoring sessions, including details such as the date, subject matter, student feedback, and specific challenges faced during the session. This memory helps the agents recall past interactions and improve future sessions based on previous experiences.',
    '# semantic ## subject_knowledge ### A structured repository of knowledge on various subjects, including definitions, key concepts, and common problems encountered in tutoring. This memory allows agents to provide accurate information and explanations to students.',
    '# procedural ## tutoring_process ### A step-by-step guide outlining the approach agents should take when conducting a tutoring session, including preparation, delivery of content, engagement strategies, and assessment methods. This memory ensures a consistent and effective tutoring experience across all agents.',
    '# prospective ## future_goals ### A list of objectives and milestones for both students and agents, including skill acquisition targets, progress tracking, and plans for future sessions. This memory helps agents stay focused on the educational journey of their students and encourages proactive planning.',
]
run_memory_generator(interaction='I created', memory_schema=memory_schema)

Memory Generated:
-----------------
# tutoring_sessions  
## Memory Content:  
1. **Date:** March 15, 2023  
   **Subject Matter:** Algebra  
   **Student Feedback:** "I felt more confident with quadratic equations after this session."  
   **Challenges Faced:** The student struggled with factoring, which required additional practice and explanations.  

2. **Date:** March 22, 2023  
   **Subject Matter:** Geometry  
   **Student Feedback:** "The visual aids really helped me understand the concepts better."  
   **Challenges Faced:** The student had difficulty with theorems related to circles, which necessitated a new approach using diagrams.  

3. **Date:** March 29, 2023  
   **Subject Matter:** Calculus  
   **Student Feedback:** "I appreciated the step-by-step breakdown of limits."  
   **Challenges Faced:** The student needed extra time to grasp the concept of continuity, prompting additional examples and practice problems.  

### memory_context  
This memory serves to enhance fut